In [1]:
import jieba;
jieba.enable_paddle();
#ya,I think jieba are kinda magical

import copy;
import math;
import json;
import requests;
from random import randrange
from bs4 import BeautifulSoup;



things={};
with open("./output415-Copy1.txt",encoding='utf-8') as ifile:
    things=json.loads(ifile.read());
print("done!");

Paddle enabled successfully......


done!


In [2]:
print(things["data"][1]);

{'name': '蝙蝠俠', 'link': 'https://movies.yahoo.com.tw/movieinfo_main/%E8%9D%99%E8%9D%A0%E4%BF%A0-the-batman-10904', 'when': '2022-03-04', 'desc': '《蝙蝠俠》將聚焦於一個年輕版本的布魯斯韋恩，並從中檢視他在面罩之下的靈魂。羅伯派汀森將在新版《蝙蝠俠》中飾演這位DC超級英雄，電影是由《猩球崛起：終極決戰》導演麥特李維斯（Matt Reeves）所執導。', 'class': ['動作', '犯罪']}


In [70]:

def scan_doc(inname,inlist):
    doc={
        "name":inname,
        "total_word_count":0,
        "stat":{}# word -> {count,idf}
    };
    
    #scan thorogh list
    for word in inlist:
        if word in doc["stat"]:
            doc["stat"][word]["count"]+=1;
        else:
            doc["stat"][word]={};
            doc["stat"][word]["name"]=word;
            doc["stat"][word]["source"]=inname;
            doc["stat"][word]["count"]=1;
        doc["total_word_count"]+=1;
    #end of loop
   
    return doc;
#end of scan_doc

def scan_doc_list(inlist):
    docs={
        "total_doc_count":0,
        "total_word_count":0,
        "stat":{},# word -> {count,idf}
        "doc":{}#line -> doc
    };
    
    #scan every doc in lista
    for line,line_of_list in inlist.items():
        doc=scan_doc(line,line_of_list);
        docs["doc"][line]=doc;
        #merge stat loop
        for word,info in doc["stat"].items():
            if word in docs["stat"]:
                docs["stat"][word]["count"]+=doc["stat"][word]["count"];
                docs["stat"][word]["doc_count"]+=1;
            else:
                docs["stat"][word]={};
                docs["stat"][word]["name"]=word;
                docs["stat"][word]["count"]=copy.deepcopy(doc["stat"][word]["count"]);
                docs["stat"][word]["doc_count"]=1;
        #end of merge loop
        docs["total_doc_count"]+=1;
        docs["total_word_count"]+=doc["total_word_count"];
    #end of scan
    
    for word,info in docs["stat"].items():
        info["idf"]=math.log((docs["total_doc_count"])/(info["doc_count"]),10);
    #end of idf cal
    
    for word,doc in docs["doc"].items():
        for word,info in doc["stat"].items():
            tr=(info["count"])/(doc["total_word_count"]);
            idf=docs["stat"][word]["idf"];
            info["tfidf"]=(tr)*(idf);
        #end of a word
    #end of a doc
    return docs;
#end of scan_doc_list



def preset_vector(invec1):
    def cal(invec2):
        range=0;
        for key in invec1.keys():
            if key in invec2.keys():
                range+=((invec1[key]-invec2[key])**2)
        return math.sqrt(range);
    return cal;
#end of preset_vector

def sort_and_gen_dataset(rawdata,inlambda,insize):
    return sorted(rawdata,reverse=True,key=inlambda)[0:insize];
#end of sort_and_gen_dataset



class prediction:
    def __init__(self,indataset):
        self.vector={};#each contain word,index and their idf value
        self.map=[];#each contain list of tfidf,and class
        
        doc_hash={}
        doc_list={};
        for i,entry in enumerate(indataset):
            line=entry["name"]+entry["desc"];
            cut_list=list(jieba.cut(entry["name"]))+list(jieba.cut(entry["desc"]));
            doc_list[line]=cut_list;
            doc_hash[line]=i;
        #end of making thing from list
        
        #yes,code resused
        docs=scan_doc_list(doc_list);
        
        #converter them into vector
        for entry in docs["stat"].values():
            self.vector[entry["name"]]={"idf":entry["idf"]};
        
        for entry in docs["doc"].values():
            class_list=indataset[doc_hash[entry["name"]]]["class"];
            point={"vector":{},"class":class_list};
            for word,info in entry["stat"].items():
                point["vector"][word]=info["tfidf"];
            self.map.append(point);
                
        #print(self.vector);
        #print(self.map);
        
        print("training done!")
        return;
    def predict(self,inline):
        word_list=list(jieba.cut(inline["name"]))+list(jieba.cut(inline["desc"]));
        #print(word_list);
        
        #scan
        doc={
            "total_word_count":0,
            "stat":{}# word -> {count,idf}
        };
        for word in word_list:
            if word in doc["stat"]:
                doc["stat"][word]["count"]+=1;
            else:
                doc["stat"][word]={};
                doc["stat"][word]["name"]=word;
                doc["stat"][word]["count"]=1;
            doc["total_word_count"]+=1;
        #end of loop
        #print(doc);
        
        #tridf
        result_vector={};
        for key in self.vector.keys():
            if key in doc["stat"].keys():
                tr=(doc["stat"][key]["count"])/(doc["total_word_count"]);
                idf=self.vector[key]["idf"];
                result_vector[key]=(tr)*(idf);
        #end of a word
        #print(result_vector);
        
        #sort and getting the
        cal=preset_vector(result_vector);
        sed=sort_and_gen_dataset(self.map,lambda i:cal(i["vector"]),100);
        fullclass={};
        for entry in sed:
            for tag in entry["class"]:
                if tag in fullclass.keys():
                    fullclass[tag]["count"]+=1;
                else:
                    fullclass[tag]={"index":len(fullclass),"name":tag,"count":1};
        sed_of_fc=sort_and_gen_dataset(fullclass.values(),lambda i:i["count"],6);
        #print(sed_of_fc);
        
        result=[entry["name"] for entry in sed_of_fc];
        return result;
    #end of __init
#end of prediction

train=prediction(things["data"]);

training done!


In [72]:
#this is part for you to do the testing
import csv

dataset=[]
with open('dataset.csv',encoding='utf_8_sig') as f:
    dataset = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

print(dataset[0:1]);



#and this is example of doing 1 test
print(dataset[2]["name"],"setarted");
print(train.predict(dataset[2]));#this is how do it for a entry


#this is example do it for a range
for entry in dataset[15:37]:
    print(entry["name"],"setarted");
    print(train.predict(entry));
    print("");

[{'name': '小小兵2：格魯的崛起', 'link': 'https://movies.yahoo.com.tw/movieinfo_main/%E5%B0%8F%E5%B0%8F%E5%85%B52-%E6%A0%BC%E9%AD%AF%E7%9A%84%E5%B4%9B%E8%B5%B7-minions-the-rise-of-guru-10493', 'when': '2022-07-01', 'desc': '今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將帶來關於一個12歲小男孩的故事，他夢想成為世上最偉大的超級大壞蛋。這部全新動畫喜劇動作片就是《小小兵2：格魯的崛起》。\n\xa0\n這部電影充滿了照明娛樂招牌的顛覆性和無厘頭喜劇橋段、各種大眾文化的笑梗、感人肺腑的劇情、勁爆的電影配樂，以及宇宙超級無敵誇張的動作場面。《小小兵2：格魯的崛起》一片由這個電影系列的原創掌舵，製片包括照明娛樂極具遠見的創辦人和執行長克里斯梅勒丹德利、他的長期合作夥伴珍娜海莉，以及克里斯雷納德，導演凱爾巴爾達（《神偷奶爸3》、《小小兵》）則再度回歸執導這部電影。共同導演是布萊德艾柏森（長壽喜劇影集《辛普森家庭》）以及強納森戴凡（《寵物當家》系列電影）。', 'class': "['動畫', '動作', '喜劇']"}]
魔比斯 setarted
['劇情', '動作', '懸疑/驚悚', '冒險', '科幻', '恐怖']
脫稿玩家 setarted
['劇情', '動作', '冒險', '動畫', '懸疑/驚悚', '喜劇']

玩命關頭9 setarted
['劇情', '動作', '懸疑/驚悚', '喜劇', '冒險', '恐怖']

自殺突擊隊：集結 setarted
['劇情', '懸疑/驚悚', '動作', '恐怖', '喜劇', '犯罪']

除暴 setarted
['劇情', '動作', '懸疑/驚悚', '犯罪', '喜劇', '冒險']

叢林奇航 setarted
['劇情', '懸疑/驚悚', '動作', '冒險', '恐怖', '動畫']

宇宙特攻隊 setarted
['劇情', '動作', '冒險', '科幻', '動畫', '喜劇']

黑寡婦 setarted
['劇情', '動作', '懸疑/驚悚', '冒險', '恐